Anmerkung: 
habe den Code gerade zusammenkopiert (27.05.) und mache jetzt Pause, sortiere dann morgen alles um das mit der linearen regression nochmal auszuprobieren!

Anmerkung: 
Dieser Code ist in mittlerweile Teilen fertig und der Datensatz daraus kann weiterverwendet werden.
Was noch offen ist: Die saisonalen Abweichungen (Temperatur) zu berechnen und einzufügen. (Hinweis: Die Abweichungen könnten als numerische Variablen eingehen. Es müsste dann eine Variable für 'weicht positiv vom Mittelwert ab' und 'weicht negativ vom Mittelwert ab' geben, damit man dem Modell die Beträge der Abweichung eingeben kann und nicht mit negative Zahlen handhaben muss. Das alles wäre schön zu haben, aber erstmal müssen wr den Workflow anpassen und wenn am Ende noch Zeit über ist, dann vielleicht...)


In [3]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import csv

In [4]:
# load data: is "5_has_wettercode" the latest version?

df_wetter = pd.read_csv("/workspaces/bakery_sales_prediction/sourcedata/cleaned_data/5_has_wettercode.csv")
print(df_wetter.head(), "\n")

        Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
0  2013-07-01         6.0     17.8375                 15.0        20.0   
1  2013-07-01         6.0     17.8375                 15.0        20.0   
2  2013-07-01         6.0     17.8375                 15.0        20.0   
3  2013-07-01         6.0     17.8375                 15.0        20.0   
4  2013-07-01         6.0     17.8375                 15.0        20.0   

   KielerWoche  Warengruppe      Umsatz  Feiertage  Ferientage  
0          0.0          1.0  148.828353        0.0         1.0  
1          0.0          2.0  535.856285        0.0         1.0  
2          0.0          3.0  201.198426        0.0         1.0  
3          0.0          4.0   65.890169        0.0         1.0  
4          0.0          5.0  317.475875        0.0         1.0   



## 1. Wettercodes:

### Wetterdcodes decodieren und in neue Klassen einteilen

In [5]:
# Print the frequency of each value of the weather codes (which ones occure?)
value_counts = df_wetter['Wettercode'].value_counts().sort_index()
print(value_counts)

Wettercode
0.0      815
3.0        5
5.0      562
10.0     742
17.0       5
20.0     244
21.0    1013
22.0      88
28.0      63
45.0      96
49.0      48
53.0      31
61.0    2271
63.0     535
65.0     111
68.0      12
69.0      27
71.0      81
73.0      48
75.0      25
77.0      45
79.0       5
95.0     137
Name: count, dtype: int64


In [6]:
# Dictionary with weather codes and their descriptions from http://www.seewetter-kiel.de/seewetter/daten_symbole.htm
weather_code_descriptions = {
    0: "Bewoelkungsentwicklung nicht beobachtet",
    3: "Zunehmende Bewoelkung",
    5: "Trockener Dunst",
    10: "Feuchter Dunst",
    17: "Gewitter mit hoerbarem Donner, aber kein Niederschlag an der Station",
    20: "Spruehregen oder Schneegriesel hat aufgehoert",
    21: "Regen hat aufgehoert",
    22: "Schneefall hat aufgehoert",
    28: "Nebel hat sich aufgeloest",
    45: "Nebel, Himmel ist nicht erkennbar, Nebel unveraendert",
    49: "Nebel mit Reifbildung, Himmel ist nicht erkennbar",
    53: "Durchgehender maeßiger und nicht gefrierender Spruehregen",
    61: "Durchgehender leichter und nicht gefrierender Regen",
    63: "Durchgehender maeßiger nicht gefrierender Regen",
    65: "Durchgehender starker und nicht gefrierender Regen",
    68: "Leichter Schneeregen",
    69: "Maeßiger oder starker Schneeregen",
    71: "Durchgehender leichter Schneefall",
    73: "Durchgehender maeßiger Schneefall",
    75: "Durchgehender starker Schneefall",
    77: "Schneegrieseln mit oder ohne Nebel",
    79: "Eiskoerner (gefrorene Regentropfen)",
    95: "Leichtes oder maeßiges Gewitter mit Regen oder Schnee"
}

# Function to get the description of a weather code from list above
def get_weather_description(code):
    return weather_code_descriptions.get(code, "Description not available")

get_weather_description(53)

'Durchgehender maeßiger und nicht gefrierender Spruehregen'

In [7]:
# new classes for weathercodes in a dictionary:
new_weather_code_descriptions = {
    0 : "nicht_beobachtet",
    1 : "Wolken",
    2 : "Dunst_oder_Nebel",
    3 : "sich_ankuendigender_Regen",
    4 : "Gewitter",
    5 : "Boeen_und_Sturm",
    6 : "Niederschlag_nass_leicht", # Regen # leicht # inkl. 'Niederschlag hat aufgehört', weil es ja nicht den ganzen Tag lang 'aufgehört' haben kann, sondern dann vermutlich vorher geregnet hat
    7 : "Niederschlag_nass_stark", # mäßiger oder starker Regen
    8 : "Niederschlag_trocken", # Schnee; leichtes oder mäßiges Schneetreiben
    9 : "Schneesturm_Hagel", # starkes Schneefegen
    10 : "Sandsturm"
}

# Define lists of codes for each weather class
nicht_beobachtet =  [0] # 0
Wolken = [1, 2, 3] # 1
Dunst_oder_Nebel = [4, 5, 6, 7, 8, 9, 10, 11, 12, 28, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49] # 2
sich_ankuendigender_Regen = [14, 15, 16] # 3
Gewitter = [13, 17, 29, 91, 92, 93, 94, 95, 96, 97, 98, 99] # 4
Boeen_und_Sturm = [18, 19] # 5
Niederschlag_nass_leicht = [20, 21, 23, 24, 25, 27, 50, 51, 56, 58, 60, 61, 66, 68, 80, 83, 87] # 6
Niederschlag_nass_stark = [52, 53, 54, 55, 57, 59, 62, 63, 64, 65, 67, 69, 81, 82, 84, 88] # 7
Niederschlag_trocken = [22, 26, 36, 38, 70, 71, 72, 73, 74, 75, 77, 78, 85, 86] # 8
Schneesturm_Hagel = [37, 39, 76, 79, 89, 90] # 9
Sandsturm = [30, 31, 32, 33, 34, 35] # 10


In [8]:
# Create a combined dictionary for mapping original codes to new numeric codes
weather_code_mapping = {}

# Combine the lists and dictionary
weather_classes = {
    "nicht_beobachtet" : nicht_beobachtet,
    "Wolken" : Wolken,
    "Dunst_oder_Nebel" : Dunst_oder_Nebel,
    "sich_ankuendigender_Regen" : sich_ankuendigender_Regen,
    "Gewitter" : Gewitter,
    "Boeen_und_Sturm" : Boeen_und_Sturm,
    "Niederschlag_nass_leicht" : Niederschlag_nass_leicht,
    "Niederschlag_nass_stark" : Niederschlag_nass_stark,
    "Niederschlag_trocken" : Niederschlag_trocken,
    "Schneesturm_Hagel" : Schneesturm_Hagel,
    "Sandsturm" : Sandsturm
}

# Iterate over the new_weather_code_descriptions to build the mapping dictionary
for new_code, description in new_weather_code_descriptions.items():
    if description in weather_classes:
        for code in weather_classes[description]:
            weather_code_mapping[code] = new_code

# Debugging: Print the mapping dictionary to ensure it is correct
print("Weather Code Mapping:", weather_code_mapping)

# Map the weather codes to the new numeric codes
df_wetter['Wetterklasse'] = df_wetter['Wettercode'].map(weather_code_mapping)

# Check for rows where 'Wetterklasse' is NaN
missing_wetterklasse = df_wetter[df_wetter['Wetterklasse'].isna()]

# If there are any missing 'Wetterklasse', print those rows
if not missing_wetterklasse.empty:
    print("Rows with missing 'Wetterklasse':")
    print(missing_wetterklasse)
else:
    print("No rows with missing 'Wetterklasse'.")

# Replace NaN values with -9999 for codes not in the mapping
#df_wetter['Wetterklasse'] = df_wetter['Wetterklasse'].fillna(-9999)

# If the new column is still not appearing, check the DataFrame's columns
print("DataFrame Columns:", df_wetter.columns)
# Print the updated DataFrame
print(df_wetter)

Weather Code Mapping: {0: 0, 1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 28: 2, 40: 2, 41: 2, 42: 2, 43: 2, 44: 2, 45: 2, 46: 2, 47: 2, 48: 2, 49: 2, 14: 3, 15: 3, 16: 3, 13: 4, 17: 4, 29: 4, 91: 4, 92: 4, 93: 4, 94: 4, 95: 4, 96: 4, 97: 4, 98: 4, 99: 4, 18: 5, 19: 5, 20: 6, 21: 6, 23: 6, 24: 6, 25: 6, 27: 6, 50: 6, 51: 6, 56: 6, 58: 6, 60: 6, 61: 6, 66: 6, 68: 6, 80: 6, 83: 6, 87: 6, 52: 7, 53: 7, 54: 7, 55: 7, 57: 7, 59: 7, 62: 7, 63: 7, 64: 7, 65: 7, 67: 7, 69: 7, 81: 7, 82: 7, 84: 7, 88: 7, 22: 8, 26: 8, 36: 8, 38: 8, 70: 8, 71: 8, 72: 8, 73: 8, 74: 8, 75: 8, 77: 8, 78: 8, 85: 8, 86: 8, 37: 9, 39: 9, 76: 9, 79: 9, 89: 9, 90: 9, 30: 10, 31: 10, 32: 10, 33: 10, 34: 10, 35: 10}
No rows with missing 'Wetterklasse'.
DataFrame Columns: Index(['Datum', 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit',
       'Wettercode', 'KielerWoche', 'Warengruppe', 'Umsatz', 'Feiertage',
       'Ferientage', 'Wetterklasse'],
      dtype='object')
           Datum  Bewoel

In [9]:
print("describe: \n", df_wetter.describe(), "\n")

print("NaN: \n", df_wetter.isnull().value_counts(), "\n") # how to count the isnull values?

describe: 
         Bewoelkung   Temperatur  Windgeschwindigkeit   Wettercode  \
count  7009.000000  7009.000000          7009.000000  7009.000000   
mean      5.307319    10.726434            11.174775    36.900128   
std       2.394701     6.843793             4.332547    27.095884   
min       0.000000    -8.475000             3.000000     0.000000   
25%       4.000000     5.500000             8.000000    10.000000   
50%       6.000000    10.000000            10.000000    28.000000   
75%       7.000000    16.387500            14.000000    61.000000   
max       8.000000    28.875000            35.000000    95.000000   

       KielerWoche  Warengruppe       Umsatz    Feiertage   Ferientage  \
count  7009.000000  7009.000000  7009.000000  7009.000000  7009.000000   
mean      0.023541     3.100014   200.362704     0.019404     0.152233   
std       0.151625     1.497091   140.506394     0.137949     0.359272   
min       0.000000     1.000000     7.051201     0.000000     0.000000

### Klassen mit zu wenig Objekten herausfiltern

In [22]:
# Find a frequency threshold for frequent classes (which ones to include?) to reduce noise (small object numbers per class might be too little to include into the model --> find threshold)

# Calculate value counts
value_counts = df_wetter['Wetterklasse'].value_counts().sort_index()

# Calculate total number of occurrences
total_counts = value_counts.sum()

# Define the threshold percentage (e.g., 0.5, 1, 2, 5%)
threshold_percentage = 0.01

# Calculate the minimum number of occurrences to be included
threshold_count = total_counts * threshold_percentage

# Filter weather codes based on the threshold count and only include the 'valid'
frequent_classes = value_counts[value_counts >= threshold_count]

print("Total occurrences per weather code:")
print(value_counts)
print(f"\nFiltered weather codes (above threshold {threshold_percentage}):")
print(frequent_classes)
# Pretty print the mapping dictionary
print("\nReminder: Weather Code Mapping:")
for key, value in new_weather_code_descriptions.items():
    print(f"{key}: {value}")


Total occurrences per weather code:
Wetterklasse
0     815
1       5
2    1511
4     142
6    3540
7     704
8     287
9       5
Name: count, dtype: int64

Filtered weather codes (above threshold 0.01):
Wetterklasse
0     815
2    1511
4     142
6    3540
7     704
8     287
Name: count, dtype: int64

Reminder: Weather Code Mapping:
0: nicht_beobachtet
1: Wolken
2: Dunst_oder_Nebel
3: sich_ankuendigender_Regen
4: Gewitter
5: Boeen_und_Sturm
6: Niederschlag_nass_leicht
7: Niederschlag_nass_stark
8: Niederschlag_trocken
9: Schneesturm_Hagel
10: Sandsturm


In [23]:
# Datensatz anpassen und rows löschen? 
# ist es wichtiger, möglichst viele Daten drin zu haben, oder wichtiger die, mit nicht oft vorkommenden Wetterereignissen rauszulassen?

# Keep only rows with frequent Wetterklasse
df_wetter_filtered = df_wetter[df_wetter['Wetterklasse'].isin(frequent_classes.index)]
print(df_wetter_filtered.describe())

        Bewoelkung  Temperatur  Windgeschwindigkeit   Wettercode  KielerWoche  \
count  6999.000000  6999.00000          6999.000000  6999.000000  6999.000000   
mean      5.308473    10.72907            11.175025    36.894271     0.023575   
std       2.392562     6.84571             4.334977    27.076717     0.151731   
min       0.000000    -8.47500             3.000000     0.000000     0.000000   
25%       4.000000     5.50000             8.000000    10.000000     0.000000   
50%       6.000000    10.00000            10.000000    28.000000     0.000000   
75%       7.000000    16.38750            14.000000    61.000000     0.000000   
max       8.000000    28.87500            35.000000    95.000000     1.000000   

       Warengruppe       Umsatz    Feiertage   Ferientage  Wetterklasse  
count  6999.000000  6999.000000  6999.000000  6999.000000   6999.000000  
mean      3.100157   200.378479     0.019431     0.152450      4.579797  
std       1.497201   140.549060     0.138045    

In [ ]:
# output after previous steps: 
df_wetter_filtered

## 2. Niederschlagsdaten einfügen

In [24]:
df_weather = df_wetter_filtered

In [39]:
# read in weather data
#weather = pd.read_csv("/workspaces/bakery_sales_prediction/sourcedata/wetter.csv")
df_weather["Datum"] = pd.to_datetime(df_weather["Datum"])

# Define the path to additional weather data from DWD
file_path = "/workspaces/bakery_sales_prediction/sourcedata/produkt_klima_tag_19740101_20231231_02564.txt"

# Read the file into a DataFrame, parse dates, and treat -999 as NaN
df_prec = pd.read_csv(file_path, sep=';', na_values='-999', parse_dates=['MESS_DATUM'])
df_prec["MESS_DATUM"] = pd.to_datetime(df_prec["MESS_DATUM"])
print(f"Niederschlagsdaten: \n{df_prec.head()}")

# Rename the MESS_DATUM column to Datum
df_prec.rename(columns={'MESS_DATUM': 'Datum'}, inplace=True)
df_prec.rename(columns={" RSK": "Niederschlag"}, inplace = True)
# prec.rename(columns={"  FM": "Windgeschwindigkeit_m-per-s"}, inplace = True)

# Select relevant columns
df_prec = df_prec[["Datum", "Niederschlag"]]

# Display the first few rows of the DataFrame
print(f"\nNiederschlagsdaten Auswahl: \n{df_prec.head()}")

# Merge both datasets on the Datum column
df_wetter_n = pd.merge(df_weather, df_prec, on='Datum', how='inner')

# Display the merged DataFrame
print(f"\nNeuer df Wetter mit Niederschlag: \n{df_wetter_n}")

# check for statistics:
print(f"\nStatistics for precipitation: \n{df_wetter_n['Niederschlag'].describe()}")

Niederschlagsdaten: 
   STATIONS_ID MESS_DATUM  QN_3    FX    FM  QN_4   RSK  RSKF   SDK  SHK_TAG  \
0         2564 1974-01-01   5.0   6.2   2.7   NaN   NaN   NaN   NaN      NaN   
1         2564 1974-01-02   5.0   7.7   3.2   NaN   NaN   NaN   NaN      NaN   
2         2564 1974-01-03   5.0   7.2   3.2   NaN   NaN   NaN   NaN      NaN   
3         2564 1974-01-04   5.0   6.7   3.5   NaN   NaN   NaN   NaN      NaN   
4         2564 1974-01-05   5.0   9.2   3.4   NaN   NaN   NaN   NaN      NaN   

     NM   VPM    PM   TMK   UPM   TXK   TNK   TGK  eor  
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  eor  
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  eor  
2   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  eor  
3   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  eor  
4   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  eor  

Niederschlagsdaten Auswahl: 
       Datum  Niederschlag
0 1974-01-01           NaN
1 1974-01-02           NaN
2 1974-01-03           NaN
3 1974-01-04      

/tmp/ipykernel_10748/1766985759.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weather["Datum"] = pd.to_datetime(df_weather["Datum"])


In [40]:
# output after the previous steps:
df_wetter_n

,Datum,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche,Warengruppe,Umsatz,Feiertage,Ferientage,Wetterklasse,Niederschlag
0,2013-07-01,6.0,17.8375,15.0,20.0,0.0,1.0,148.828353,0.0,1.0,6,0.3
1,2013-07-01,6.0,17.8375,15.0,20.0,0.0,2.0,535.856285,0.0,1.0,6,0.3
2,2013-07-01,6.0,17.8375,15.0,20.0,0.0,3.0,201.198426,0.0,1.0,6,0.3
3,2013-07-01,6.0,17.8375,15.0,20.0,0.0,4.0,65.890169,0.0,1.0,6,0.3
4,2013-07-01,6.0,17.8375,15.0,20.0,0.0,5.0,317.475875,0.0,1.0,6,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...
6994,2018-07-30,6.0,27.7875,10.0,21.0,0.0,1.0,148.541340,0.0,1.0,6,0.0
6995,2018-07-30,6.0,27.7875,10.0,21.0,0.0,2.0,616.358562,0.0,1.0,6,0.0
6996,2018-07-30,6.0,27.7875,10.0,21.0,0.0,3.0,348.770346,0.0,1.0,6,0.0
6997,2018-07-30,6.0,27.7875,10.0,21.0,0.0,4.0,71.822977,0.0,1.0,6,0.0


## 3. Wochentage

In [41]:
df_w = df_wetter_n

In [42]:
# extract date features
df_w['year'] = df_w['Datum'].dt.year
df_w['month'] = df_w['Datum'].dt.month
df_w['week'] = df_w['Datum'].dt.isocalendar().week
df_w['weekday'] = df_w['Datum'].dt.dayofweek
df_w['day_month'] = df_w['Datum'].dt.day
print(df_w.head())


       Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
0 2013-07-01         6.0     17.8375                 15.0        20.0   
1 2013-07-01         6.0     17.8375                 15.0        20.0   
2 2013-07-01         6.0     17.8375                 15.0        20.0   
3 2013-07-01         6.0     17.8375                 15.0        20.0   
4 2013-07-01         6.0     17.8375                 15.0        20.0   

   KielerWoche  Warengruppe      Umsatz  Feiertage  Ferientage  Wetterklasse  \
0          0.0          1.0  148.828353        0.0         1.0             6   
1          0.0          2.0  535.856285        0.0         1.0             6   
2          0.0          3.0  201.198426        0.0         1.0             6   
3          0.0          4.0   65.890169        0.0         1.0             6   
4          0.0          5.0  317.475875        0.0         1.0             6   

   Niederschlag  year  month  week  weekday  day_month  
0           0.3  2013  

In [43]:
print(df_w["month"].describe())

count    6999.000000
mean        6.581226
std         3.670393
min         1.000000
25%         3.000000
50%         7.000000
75%        10.000000
max        12.000000
Name: month, dtype: float64


In [44]:
# define the season
def get_season(month):
	    if month in [12, 1, 2]:
	        return 'winter', 4
	    elif month in [3, 4, 5]:
	        return 'spring', 1
	    elif month in [6, 7, 8]:
	        return 'summer', 2
	    else:
	        return 'autumn', 3
	
df_w['season_str'], df_w['season'] = zip(*df_w['month'].apply(get_season))


# Create binary features for seasons --> creates one column per season and then 0/1
# probably to generic for predictions?

#weather_data['winter'] = (weather_data['season'] == 'winter').astype(int)
#weather_data['spring'] = (weather_data['season'] == 'spring').astype(int)
#weather_data['summer'] = (weather_data['season'] == 'summer').astype(int)
#weather_data['autumn'] = (weather_data['season'] == 'autumn').astype(int)


print(df_w.head())


       Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
0 2013-07-01         6.0     17.8375                 15.0        20.0   
1 2013-07-01         6.0     17.8375                 15.0        20.0   
2 2013-07-01         6.0     17.8375                 15.0        20.0   
3 2013-07-01         6.0     17.8375                 15.0        20.0   
4 2013-07-01         6.0     17.8375                 15.0        20.0   

   KielerWoche  Warengruppe      Umsatz  Feiertage  Ferientage  Wetterklasse  \
0          0.0          1.0  148.828353        0.0         1.0             6   
1          0.0          2.0  535.856285        0.0         1.0             6   
2          0.0          3.0  201.198426        0.0         1.0             6   
3          0.0          4.0   65.890169        0.0         1.0             6   
4          0.0          5.0  317.475875        0.0         1.0             6   

   Niederschlag  year  month  week  weekday  day_month season_str  season  
0   

In [45]:
# dataframe after the previous steps:
df_wetter_seasons = df_w

df_wetter_seasons

,Datum,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche,Warengruppe,Umsatz,Feiertage,Ferientage,Wetterklasse,Niederschlag,year,month,week,weekday,day_month,season_str,season
0,2013-07-01,6.0,17.8375,15.0,20.0,0.0,1.0,148.828353,0.0,1.0,6,0.3,2013,7,27,0,1,summer,2
1,2013-07-01,6.0,17.8375,15.0,20.0,0.0,2.0,535.856285,0.0,1.0,6,0.3,2013,7,27,0,1,summer,2
2,2013-07-01,6.0,17.8375,15.0,20.0,0.0,3.0,201.198426,0.0,1.0,6,0.3,2013,7,27,0,1,summer,2
3,2013-07-01,6.0,17.8375,15.0,20.0,0.0,4.0,65.890169,0.0,1.0,6,0.3,2013,7,27,0,1,summer,2
4,2013-07-01,6.0,17.8375,15.0,20.0,0.0,5.0,317.475875,0.0,1.0,6,0.3,2013,7,27,0,1,summer,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6994,2018-07-30,6.0,27.7875,10.0,21.0,0.0,1.0,148.541340,0.0,1.0,6,0.0,2018,7,31,0,30,summer,2
6995,2018-07-30,6.0,27.7875,10.0,21.0,0.0,2.0,616.358562,0.0,1.0,6,0.0,2018,7,31,0,30,summer,2
6996,2018-07-30,6.0,27.7875,10.0,21.0,0.0,3.0,348.770346,0.0,1.0,6,0.0,2018,7,31,0,30,summer,2
6997,2018-07-30,6.0,27.7875,10.0,21.0,0.0,4.0,71.822977,0.0,1.0,6,0.0,2018,7,31,0,30,summer,2


## 4. Saisonale Abweichungen:

In [ ]:
# check what exactly this means and does! (Suggestion by ChatGPT)
# Create rolling averages for temperature, precipitation and and wind speed
# weather_data['temp_7d_avg'] = weather_data['Temperatur'].rolling(window=7).mean()
# weather_data['wind_7d_avg'] = weather_data['Windgeschwindigkeit'].rolling(window=7).mean()
# weather_data['prec_7d_avg'] = weather_data['Niederschlag'].rolling(window=7).mean()

# Create lag features
# weather_data['temp_lag_1'] = weather_data['Temperatur'].shift(1)
# weather_data['wind_lag_1'] = weather_data['Windgeschwindigkeit'].shift(1)
# weather_data['prec_lag_1'] = weather_data['Niederschlag'].shift(1)

# Fill NA values created by rolling and lag features
# weather_data.fillna(method='bfill', inplace=True)

# Normalize the weather features
# from sklearn.preprocessing import StandardScaler

# weather_features = ['Temperatur', 'Windgeschwindigkeit', 'Niederschlag', 'temp_7d_avg', 'wind_7d_avg', 'temp_lag_1', 'wind_lag_1', 'prec_7d_avg', 'prec_lag_1']
# scaler = StandardScaler()
# weather_data[weather_features] = scaler.fit_transform(weather_data[weather_features])
# print(weather_data.head())


In [ ]:
# Open ToDos:
# Create categories for temperature depending on season
# or calculate the deviation from monthly temperature
# or define warm / cold day depending on the season (e.g. in winter 10°C is a warm day, in summer that would be a cold day)

In [ ]:
# Define thresholds for extreme weather conditions
VERY_HOT_THRESHOLD = 25  # Example: temperature > 25 degrees Celsius
VERY_COLD_THRESHOLD = 0  # Example: temperature < 0 degrees Celsius
VERY_RAINY_THRESHOLD = 10 # Example: precipitation > 10 mm
HIGH_WIND_THRESHOLD = 15  # Example: wind speed > 15 m/s
SUNNY_DAY_THRESHOLD = 2  # Example: cloudiness <= 2 - how is the parameter defined?

# Create binary features for extreme weather conditions
weather_data['very_hot'] = (weather_data['Temperatur'] > VERY_HOT_THRESHOLD).astype(int)
weather_data['very_cold'] = (weather_data['Temperatur'] < VERY_COLD_THRESHOLD).astype(int)
weather_data['very_rainy'] =  (weather_data['Niederschlag'] > VERY_RAINY_THRESHOLD).astype(int)
weather_data['high_wind'] = (weather_data['Windgeschwindigkeit'] > HIGH_WIND_THRESHOLD).astype(int)
weather_data['sunny_days'] = (weather_data['Bewoelkung'] <= SUNNY_DAY_THRESHOLD).astype(int)

# Display the first few rows to check all features
print(weather_data)


In [ ]:
# Calculate "feels like" temperature
weather_data["Windgeschwindigkeit_km_h"] = weather_data["Windgeschwindigkeit"] * 3.6
weather_data["feels_like_temperature"] = 13.12 + 0.6215 * weather_data["Temperatur"] - 11.37 * weather_data["Windgeschwindigkeit_km_h"]**0.16 + 0.3965*weather_data["Temperatur"]*weather_data["Windgeschwindigkeit_km_h"]**0.16
print(weather_data.head())

# export file again
# weather_data.to_csv("Files/weather_data.csv", index = False)


## Ergebnis speichern:

In [46]:
 # Save the updated data to new CSV files
df_w.to_csv('../sourcedata/cleaned_data/6_weather_parameters.csv', sep=',', index=False)